<a href="https://colab.research.google.com/github/EduHdzVillasana/COVIDSTATSMX/blob/master/An%C3%A1lisis%20y%20Modelado/Modelo_Predictorio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creación de modelo para predecir probabilidad de muerte por COVID-19
En este notebook se creará un modelo de regresión logística para predecir la probabilidad de muerte de un paciente positivo a sars-cov-2 dadas sus condiciones.

También se creará un modelo para predecir la probabilidad de contraer enfermedad grave dadas sus condiciones y así identificar personas vulnerables.

## Librerías a utilizar


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os

# Librerias para la regresión logística
import statsmodels.api as sm
from sklearn import datasets
from sklearn.feature_selection import RFE
from tabulate import tabulate
from sklearn import linear_model

# Librerias para la validación del modelo
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
mainpath = "/content/drive/MyDrive/Datos Abiertos"
filepath = "210401COVID19MEXICO_filtered.csv"

data = pd.read_csv(os.path.join(mainpath, filepath), index_col=0)
data.sample(3)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (39) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,INTUBADO,NEUMONIA,EDAD,NACIONALIDAD,EMBARAZO,HABLA_LENGUA_INDIG,INDIGENA,DIABETES,EPOC,ASMA,INMUSUPR,HIPERTENSION,OTRA_COM,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI,RESULTADO,TIEMPO_ESPERA,FALLECIDO,ENFERMEDAD_GRAVE,HOSPITALIZADO,SEXO_,SECTOR_ATENDIDO,ESTADO
648809,2021-04-01,3f1a0f,2,4,9,1,9,9,15,2,2020-12-10,2020-12-09,9999-99-99,0,0,62,1,0,2,2,0,0,0,0,0,0,0,0,0,0,2,1,1,2,97,3,99,México,97,0,1,1,0,0,1,MUJER,IMSS,DF
1563328,2021-04-01,d3eecd,1,6,20,1,20,20,83,1,2021-03-02,2021-02-27,9999-99-99,0,0,38,1,0,2,2,0,0,1,0,0,0,0,0,0,0,2,1,1,2,97,3,99,México,97,0,1,3,0,0,0,MUJER,ISSSTE,OC
1173023,2021-04-01,1bc868,1,12,13,1,13,13,48,1,2020-08-06,2020-08-03,9999-99-99,0,0,29,1,0,2,2,0,0,0,0,0,0,0,0,0,0,1,1,1,2,97,3,99,México,97,0,1,3,0,0,0,MUJER,SSA,HG


In [3]:
columns = ['EDAD', 'EMBARAZO','DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'TIEMPO_ESPERA', 'HOSPITALIZADO','SEXO_','SECTOR_ATENDIDO','ESTADO','ENFERMEDAD_GRAVE','FALLECIDO']
data = data[columns]

## Preparación de datos

In [4]:
data = data.rename({"SEXO_":"SEXO"}, axis = 1)

In [5]:
data.columns

Index(['EDAD', 'EMBARAZO', 'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR',
       'HIPERTENSION', 'OTRA_COM', 'CARDIOVASCULAR', 'OBESIDAD',
       'RENAL_CRONICA', 'TABAQUISMO', 'TIEMPO_ESPERA', 'HOSPITALIZADO', 'SEXO',
       'SECTOR_ATENDIDO', 'ESTADO', 'ENFERMEDAD_GRAVE', 'FALLECIDO'],
      dtype='object')

### Creación de variables dummies (catagóricas)

In [6]:
categories = ["SEXO","SECTOR_ATENDIDO","ESTADO"]
for cat in categories:
  cat_dummies = pd.get_dummies(data[cat], prefix = cat)
  data = data.join(cat_dummies)

In [7]:
data.columns

Index(['EDAD', 'EMBARAZO', 'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR',
       'HIPERTENSION', 'OTRA_COM', 'CARDIOVASCULAR', 'OBESIDAD',
       'RENAL_CRONICA', 'TABAQUISMO', 'TIEMPO_ESPERA', 'HOSPITALIZADO', 'SEXO',
       'SECTOR_ATENDIDO', 'ESTADO', 'ENFERMEDAD_GRAVE', 'FALLECIDO',
       'SEXO_HOMBRE', 'SEXO_MUJER', 'SECTOR_ATENDIDO_CRUZ ROJA',
       'SECTOR_ATENDIDO_DIF', 'SECTOR_ATENDIDO_ESTATAL',
       'SECTOR_ATENDIDO_IMSS', 'SECTOR_ATENDIDO_IMSS-BIENESTAR',
       'SECTOR_ATENDIDO_ISSSTE', 'SECTOR_ATENDIDO_MUNICIPAL',
       'SECTOR_ATENDIDO_PEMEX', 'SECTOR_ATENDIDO_PRIVADA',
       'SECTOR_ATENDIDO_SEDENA', 'SECTOR_ATENDIDO_SEMAR',
       'SECTOR_ATENDIDO_SSA', 'SECTOR_ATENDIDO_UNIVERSITARIO', 'ESTADO_AS',
       'ESTADO_BC', 'ESTADO_BS', 'ESTADO_CC', 'ESTADO_CH', 'ESTADO_CL',
       'ESTADO_CM', 'ESTADO_CS', 'ESTADO_DF', 'ESTADO_DG', 'ESTADO_GR',
       'ESTADO_GT', 'ESTADO_HG', 'ESTADO_JC', 'ESTADO_MC', 'ESTADO_MN',
       'ESTADO_MS', 'ESTADO_NL', 'ESTADO_NT', 'ESTADO_OC', 'E

In [8]:
# Columnas de las condiciones del paciente ántes de ser atendido por una institución de salud
before_columns = ['EDAD', 'EMBARAZO','DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO', 'ESTADO_AS','SEXO_HOMBRE', 'SEXO_MUJER',
       'ESTADO_BC', 'ESTADO_BS', 'ESTADO_CC', 'ESTADO_CH', 'ESTADO_CL',
       'ESTADO_CM', 'ESTADO_CS', 'ESTADO_DF', 'ESTADO_DG', 'ESTADO_GR',
       'ESTADO_GT', 'ESTADO_HG', 'ESTADO_JC', 'ESTADO_MC', 'ESTADO_MN',
       'ESTADO_MS', 'ESTADO_NL', 'ESTADO_NT', 'ESTADO_OC', 'ESTADO_PL',
       'ESTADO_QR', 'ESTADO_QT', 'ESTADO_SL', 'ESTADO_SP', 'ESTADO_SR',
       'ESTADO_TC', 'ESTADO_TL', 'ESTADO_TS', 'ESTADO_VZ', 'ESTADO_YN',
       'ESTADO_ZS']

# Columnas de las condiciones del paciente después de ser atendido por una institución de salud
after_columns = ['EDAD', 'EMBARAZO','DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'TIEMPO_ESPERA', 'HOSPITALIZADO','SEXO_HOMBRE', 'SEXO_MUJER', 'SECTOR_ATENDIDO_CRUZ ROJA',
       'SECTOR_ATENDIDO_DIF', 'SECTOR_ATENDIDO_ESTATAL', 'SECTOR_ATENDIDO_IMSS', 'SECTOR_ATENDIDO_IMSS-BIENESTAR',
       'SECTOR_ATENDIDO_ISSSTE', 'SECTOR_ATENDIDO_MUNICIPAL', 'SECTOR_ATENDIDO_PEMEX', 'SECTOR_ATENDIDO_PRIVADA',
       'SECTOR_ATENDIDO_SEDENA', 'SECTOR_ATENDIDO_SEMAR', 'SECTOR_ATENDIDO_SSA', 'SECTOR_ATENDIDO_UNIVERSITARIO', 'ESTADO_AS',
       'ESTADO_BC', 'ESTADO_BS', 'ESTADO_CC', 'ESTADO_CH', 'ESTADO_CL',
       'ESTADO_CM', 'ESTADO_CS', 'ESTADO_DF', 'ESTADO_DG', 'ESTADO_GR',
       'ESTADO_GT', 'ESTADO_HG', 'ESTADO_JC', 'ESTADO_MC', 'ESTADO_MN',
       'ESTADO_MS', 'ESTADO_NL', 'ESTADO_NT', 'ESTADO_OC', 'ESTADO_PL',
       'ESTADO_QR', 'ESTADO_QT', 'ESTADO_SL', 'ESTADO_SP', 'ESTADO_SR',
       'ESTADO_TC', 'ESTADO_TL', 'ESTADO_TS', 'ESTADO_VZ', 'ESTADO_YN',
       'ESTADO_ZS']



## Modelo para condiciones iniciales del paciente


### Selección de las varianbles

In [9]:
X_before = data[before_columns].copy()
Y_deceased = data["FALLECIDO"].copy()
len(before_columns)

46

In [10]:
data.columns

Index(['EDAD', 'EMBARAZO', 'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR',
       'HIPERTENSION', 'OTRA_COM', 'CARDIOVASCULAR', 'OBESIDAD',
       'RENAL_CRONICA', 'TABAQUISMO', 'TIEMPO_ESPERA', 'HOSPITALIZADO', 'SEXO',
       'SECTOR_ATENDIDO', 'ESTADO', 'ENFERMEDAD_GRAVE', 'FALLECIDO',
       'SEXO_HOMBRE', 'SEXO_MUJER', 'SECTOR_ATENDIDO_CRUZ ROJA',
       'SECTOR_ATENDIDO_DIF', 'SECTOR_ATENDIDO_ESTATAL',
       'SECTOR_ATENDIDO_IMSS', 'SECTOR_ATENDIDO_IMSS-BIENESTAR',
       'SECTOR_ATENDIDO_ISSSTE', 'SECTOR_ATENDIDO_MUNICIPAL',
       'SECTOR_ATENDIDO_PEMEX', 'SECTOR_ATENDIDO_PRIVADA',
       'SECTOR_ATENDIDO_SEDENA', 'SECTOR_ATENDIDO_SEMAR',
       'SECTOR_ATENDIDO_SSA', 'SECTOR_ATENDIDO_UNIVERSITARIO', 'ESTADO_AS',
       'ESTADO_BC', 'ESTADO_BS', 'ESTADO_CC', 'ESTADO_CH', 'ESTADO_CL',
       'ESTADO_CM', 'ESTADO_CS', 'ESTADO_DF', 'ESTADO_DG', 'ESTADO_GR',
       'ESTADO_GT', 'ESTADO_HG', 'ESTADO_JC', 'ESTADO_MC', 'ESTADO_MN',
       'ESTADO_MS', 'ESTADO_NL', 'ESTADO_NT', 'ESTADO_OC', 'E

Nos quedaremos con la mitad de las variables 

Se utiliza el solver sag porque es más rápido para datasets grandes (en variables y tamaño de la muestra)

* [Referencia 1](https://stackoverflow.com/questions/38640109/logistic-regression-python-solvers-defintions)
* [Referencia 2](https://rubialesalberto.medium.com/regresi%C3%B3n-log%C3%ADstica-con-sklearn-4384c707075d)

In [ ]:
n = 23
lr = linear_model.LogisticRegression(max_iter=200, solver="sag")
rfe = RFE(lr,n)
rfe = rfe.fit(X_before,Y_deceased)

In [ ]:
var_included = list(zip(before_columns, rfe.support_, rfe.ranking_))
var_included.sort(key = lambda x: x[2])
print (tabulate(var_included))